# Основы построения рекомендательных систем

## Домашнее задание №3

### Основные пункты оценки
- значение метрики на лидерборде
- ревью кода в ноутбуке
- реализация сервиса для модели

### Подробности
1. Побейте метрику на лидерборде map@10 = 0.063 для userKnn модели с семинара
2. Реализуйте эксперименты с кастомной моделю kNN с семинара. Результат - ноутбук(и)

Что можно сделать в ноутбуке:
- придумать, что делать с холодными пользователями в тесте. Сделайте рекомендации для них (обратите внимание на rectools.models.popular)
- сделать кол-во рекомендаций равным N, а не меньше N
- реализовать тюнинг гиперпараметров (например, векторного расстояния или типов kNN моделей (implicit/rectools/...)) и сделать выводы
- реализовать другие варианты ранжированивания айтемов похожих пользователей и сделать выводы
- провести эксперименты с параметрами оффлайн валидации и сделать выводы

3. Оберните модель в сервис
- предпочтительный онлайн вариант: обучаете модель в ноутбуке, сохраняете обученную модель (pickle, dill), при запуске сервиса ее поднимаете и запрашиваете рекомендации "на лету"
- или оффлайн вариант: предварительно посчитайте рекомендации для всех пользователей, сохраните и запрашивайте их

### Хороший pull request - это:
- наличие описания (в идеале что сделано - по пунктам)
- код по стандарту PEP8
- легкая читаемость и воспроизводимость кода
- комментарии и объяснения. В ipynb пользуйтесь силой маркдауна. В скриптах пишите комментарии и докстринг.
- обоснование схемы валидации
- анализ метрики качества


In [1]:
# установка библиотек
import sys
!{sys.executable} -m pip install rectools
!{sys.executable} -m pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.7 MB/s eta 0:00:00


In [25]:
# подключение библиотек
import requests
from tqdm.auto import tqdm
import zipfile as zf

import numpy as np
import pandas as pd
import dill

from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import MAP, MeanInvUserFreq, NDCG, Precision, Recall, Serendipity, calc_metrics
from rectools.model_selection import TimeRangeSplitter
from implicit.nearest_neighbours import BM25Recommender, CosineRecommender, TFIDFRecommender
from models.custom_userknn import CustomUserKnn
from models.userknn import UserKnn

In [26]:
# Функция валидирования моделей методом Leave-Time-out
def validate(models: dict, metrics: dict, splitter: TimeRangeSplitter, dataset: Dataset, K_RECOS: int):
  results = []

  # создаем итератор фолдов
  fold_iterator = splitter.split(dataset.interactions, collect_fold_stats=True)

  # обучение по фолдам
  for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=splitter.n_splits):
      print(f"\n==================== Fold {fold_info['i_split']} ====================")
      print(fold_info)

      # тренировочная часть
      df_train = dataset.interactions.df.iloc[train_ids]
      train_dataset = Dataset.construct(df_train)

      # тестовая часть
      df_test = dataset.interactions.df.iloc[test_ids][Columns.UserItem]
      test_users = np.unique(df_test[Columns.User])

      # каталог
      catalog = df_train[Columns.Item].unique()


      # обучение моделей
      for model_name, model in models.items():
          # обучение и получение рекомендаций
          model.fit(train_dataset)
          recos = model.recommend(
              users=df_test,
              dataset=train_dataset,
              N_recs=K_RECOS
          )
          # подсчет метрик
          metric_values = calc_metrics(
              metrics,
              reco=recos,
              interactions=df_test,
              prev_interactions=df_train,
              catalog=catalog,
          )
          # сохранение результатов
          res = {"fold": fold_info["i_split"], "model": model_name}
          res.update(metric_values)
          results.append(res)

  # форматирование в DataFrame
  pivot_results = pd.DataFrame(results)\
                    .drop(columns="fold")\
                    .groupby(["model"], sort=False)\
                    .agg("mean")
  # сохранение отчета
  pivot_results.to_csv('./report.csv')

  return pivot_results

## Чтение данных

In [6]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='Downloading the kion dataset...',
                        total=total_size_in_bytes,
                        unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

In [7]:
# Разархивирование данных
files = zf.ZipFile('kion_train.zip','r')
files.extractall()
files.close()

In [27]:
# Чтение в DataFrame
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])
interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    },
    inplace=True)

## Валидация
**Замечание**: датасет очень большой, алгоритмы сильно ресурсно затратные. Всего 5 000 000 взаимодействий, что очень много и займет много времени для валидирования моделей. Возьмём 1 000 000 взаимодействий и проведем эксперименты.

In [39]:
interactions.shape

(5476251, 5)

In [28]:
interactions = interactions.iloc[:1000000]
dataset = Dataset.construct(interactions_df=interactions)

In [38]:
metrics = {
    "prec@1": Precision(k=1),
    "prec@5": Precision(k=5),
    "prec@10": Precision(k=10),
    "recall@1": Recall(k=1),
    "recall@5": Recall(k=5),
    "recall@10": Recall(k=10),
    "MAP@1": MAP(k=1),
    "MAP@5": MAP(k=5),
    "MAP@10": MAP(k=10),
    "NDCG@1": MAP(k=1),
    "NDCG@5": MAP(k=5),
    "NDCG@10": MAP(k=10),
    "MeanInvUserFreq@1": MeanInvUserFreq(k=1),
    "MeanInvUserFreq@5": MeanInvUserFreq(k=5),
    "MeanInvUserFreq@10": MeanInvUserFreq(k=10),
    "Serendipity@1": Serendipity(k=1),
    "Serendipity@5": Serendipity(k=5),
    "Serendipity@10": Serendipity(k=10),
}


Эксперимент №1.
- Три модели с разной мерой расстояния (Cosine, TFIDF, BM25)
- Кол-во соседей 50
- Кол-во фолдов 3
- Размер тестового фолда 7D


In [42]:
models = {
    'cosin_user_knn': CustomUserKnn(model=CosineRecommender(num_threads=2), N_users=50),
    'tfidf_user_knn': CustomUserKnn(model=TFIDFRecommender(num_threads=2), N_users=50),
    'bm25_user_knn': CustomUserKnn(model=BM25Recommender(num_threads=2), N_users=50)
    }

In [43]:
n_splits = 3
cv = TimeRangeSplitter(test_size="7D",
                        n_splits=n_splits,
                        filter_cold_users=True,
                        filter_cold_items=True,
                        filter_already_seen=True)

In [45]:
validate(models, metrics, cv, dataset, K_RECOS=10)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0 ====================
{'i_split': 0, 'start': Timestamp('2021-08-02 00:00:00', freq='7D'), 'end': Timestamp('2021-08-09 00:00:00', freq='7D'), 'train': 779557, 'train_users': 360297, 'train_items': 10978, 'test': 34230, 'test_users': 23844, 'test_items': 3793}


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.07029032707214355 seconds
  warnings.warn(


  0%|          | 0/360297 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.05586433410644531 seconds
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.06633925437927246 seconds
  warnings.warn(


  0%|          | 0/414753 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.06619668006896973 seconds
  warnings.warn(


  0%|          | 0/414753 [00:00<?, ?it/s]

,prec@1,recall@1,prec@5,recall@5,prec@10,recall@10,MAP@1,MAP@5,MAP@10,NDCG@1,NDCG@5,NDCG@10,MeanInvUserFreq@1,MeanInvUserFreq@5,MeanInvUserFreq@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,,,,,,,,,,
cosin_user_knn,0.001026,0.000813,0.000494,0.001873,0.000379,0.002858,0.000813,0.001176,0.001307,0.000813,0.001176,0.001307,11.873277,11.501205,11.407710,0.000009,0.000011,0.000011
tfidf_user_knn,0.001019,0.000819,0.000580,0.002214,0.000424,0.003141,0.000819,0.001277,0.001400,0.000819,0.001277,0.001400,11.907259,11.425128,11.304092,0.000010,0.000012,0.000012
bm25_user_knn,0.000955,0.000772,0.000383,0.001447,0.000295,0.002186,0.000772,0.001003,0.001101,0.000772,0.001003,0.001101,12.052120,11.732434,11.594274,0.000010,0.000011,0.000012


В данном эксперименте по многим метрикам выигрывает TFIDFRecommender, проигрывает только BM25Recommender'у в метрике novelty@5 и novelty@10.

Эксперимент №2.
- Три модели с разной мерой расстояния (Cosine, TFIDF, BM25)
- Кол-во соседей 10
- Кол-во фолдов 3
- Размер тестового фолда 7D


In [46]:
models = {
    'cosin_user_knn': CustomUserKnn(model=CosineRecommender(num_threads=2), N_users=10),
    'tfidf_user_knn': CustomUserKnn(model=TFIDFRecommender(num_threads=2), N_users=10),
    'bm25_user_knn': CustomUserKnn(model=BM25Recommender(num_threads=2), N_users=10)
    }

In [47]:
n_splits = 3
cv = TimeRangeSplitter(test_size="7D",
                        n_splits=n_splits,
                        filter_cold_users=True,
                        filter_cold_items=True,
                        filter_already_seen=True)

In [48]:
validate(models, metrics, cv, dataset, K_RECOS=10)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0 ====================
{'i_split': 0, 'start': Timestamp('2021-08-02 00:00:00', freq='7D'), 'end': Timestamp('2021-08-09 00:00:00', freq='7D'), 'train': 779557, 'train_users': 360297, 'train_items': 10978, 'test': 34230, 'test_users': 23844, 'test_items': 3793}


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.05373382568359375 seconds
  warnings.warn(


  0%|          | 0/360297 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.10334253311157227 seconds
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.06273365020751953 seconds
  warnings.warn(


  0%|          | 0/414753 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0614771842956543 seconds
  warnings.warn(


  0%|          | 0/414753 [00:00<?, ?it/s]

,prec@1,recall@1,prec@5,recall@5,prec@10,recall@10,MAP@1,MAP@5,MAP@10,NDCG@1,NDCG@5,NDCG@10,MeanInvUserFreq@1,MeanInvUserFreq@5,MeanInvUserFreq@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,,,,,,,,,,
cosin_user_knn,0.000986,0.000782,0.000506,0.001910,0.000330,0.002444,0.000782,0.001178,0.001253,0.000782,0.001178,0.001253,11.750056,11.411887,11.362851,0.000009,0.000011,0.000011
tfidf_user_knn,0.001019,0.000819,0.000502,0.001857,0.000323,0.002352,0.000819,0.001174,0.001243,0.000819,0.001174,0.001243,11.902942,11.483278,11.422691,0.000010,0.000011,0.000011
bm25_user_knn,0.000968,0.000785,0.000354,0.001339,0.000228,0.001701,0.000785,0.000986,0.001036,0.000785,0.000986,0.001036,11.996135,11.705930,11.647348,0.000010,0.000011,0.000011


В данном эксперименте CosineRecommender и TFIDFRecommender показывают приблизительно одинаковые резульататы, в то время как BM25Recommender проигрывает в большинстве случаев, показывая лучше результаты только по метрике novelty.\
Видно, что по сравнению с экспериментом №1, в целом у алгоритмов уменьшилась метрика MAP@10. Это происходит потому, что не было реализовано условие N-рекомендаций для каждого user'a, то есть рекомендаций у user'a может оказаться меньше того же значения 10, что сказывается на подсчитываемой метрике.

Эксперимент №3.
- Три модели с разной мерой расстояния (Cosine, TFIDF, BM25)
- Кол-во соседей 10
- Кол-во фолдов 5
- Размер тестового фолда 3D


In [49]:
models = {
    'cosin_user_knn': CustomUserKnn(model=CosineRecommender(num_threads=2), N_users=10),
    'tfidf_user_knn': CustomUserKnn(model=TFIDFRecommender(num_threads=2), N_users=10),
    'bm25_user_knn': CustomUserKnn(model=BM25Recommender(num_threads=2), N_users=10)
    }

In [50]:
n_splits = 5
cv = TimeRangeSplitter(test_size="3D",
                        n_splits=n_splits,
                        filter_cold_users=True,
                        filter_cold_items=True,
                        filter_already_seen=True)

In [51]:
validate(models, metrics, cv, dataset, K_RECOS=10)

  0%|          | 0/5 [00:00<?, ?it/s]


==================== Fold 0 ====================
{'i_split': 0, 'start': Timestamp('2021-08-08 00:00:00', freq='3D'), 'end': Timestamp('2021-08-11 00:00:00', freq='3D'), 'train': 838173, 'train_users': 382886, 'train_items': 11184, 'test': 16728, 'test_users': 13112, 'test_items': 2922}


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.05768585205078125 seconds
  warnings.warn(


  0%|          | 0/382886 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.057869672775268555 seconds
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.06680965423583984 seconds
  warnings.warn(


  0%|          | 0/429618 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.07457756996154785 seconds
  warnings.warn(


  0%|          | 0/429618 [00:00<?, ?it/s]

,prec@1,recall@1,prec@5,recall@5,prec@10,recall@10,MAP@1,MAP@5,MAP@10,NDCG@1,NDCG@5,NDCG@10,MeanInvUserFreq@1,MeanInvUserFreq@5,MeanInvUserFreq@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,,,,,,,,,,
cosin_user_knn,0.000942,0.000843,0.000448,0.001899,0.000276,0.002301,0.000843,0.001197,0.001253,0.000843,0.001197,0.001253,11.825301,11.471577,11.414977,0.000012,0.000012,0.000012
tfidf_user_knn,0.000967,0.000868,0.000433,0.001871,0.000279,0.002327,0.000868,0.001207,0.001270,0.000868,0.001207,0.001270,11.995665,11.567589,11.503152,0.000014,0.000013,0.000014
bm25_user_knn,0.000937,0.000838,0.000326,0.001428,0.000205,0.001739,0.000838,0.001025,0.001066,0.000838,0.001025,0.001066,12.093106,11.779373,11.709355,0.000014,0.000013,0.000013


В данном эксперименте CosineRecommender и TFIDFRecommender показывают приблизительно одинаковые результаты, в то время как BM25Recommender проигрывает в большинстве случаев, показывая лучше результаты только по метрике novelty.\
Метрика MAP@10 не сильно отличается от результатов, полученных в эксперименте №2, что на самом деле странно, поскольку для данных kion'a выявлена семидневная периодичность в взаимодействиях, а в опыте кроссвалидация проводилась с размером 3D для тестовой выборки.

Таким образом, в результате проведенных 3х экспериментов, можно прийти к выводу, что лучшие метрики качества показывает алгоритм TFIDFRecommender. Поэтому обучим данную модель на всем датасете из 5 млн. взаимодействий.

## Лидерборд
Проверим метрики качества модели TFIDFRecommender на всем датасете

In [10]:
interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])
interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    },
    inplace=True)

dataset = Dataset.construct(interactions_df=interactions)

In [10]:
models = { 'tfidf_user_knn': CustomUserKnn(model=TFIDFRecommender(num_threads=2), N_users=50) }

In [9]:
n_splits = 3
cv = TimeRangeSplitter(test_size="7D",
                        n_splits=n_splits,
                        filter_cold_users=True,
                        filter_cold_items=True,
                        filter_already_seen=True)

In [14]:
validate(models, metrics, cv, dataset, K_RECOS=10)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0 ====================
{'i_split': 0, 'start': Timestamp('2021-08-02 00:00:00', freq='7D'), 'end': Timestamp('2021-08-09 00:00:00', freq='7D'), 'train': 4266013, 'train_users': 797423, 'train_items': 15237, 'test': 263681, 'test_users': 98184, 'test_items': 6602}


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.44503116607666016 seconds
  warnings.warn(


  0%|          | 0/797423 [00:00<?, ?it/s]


==================== Fold 1 ====================
{'i_split': 1, 'start': Timestamp('2021-08-09 00:00:00', freq='7D'), 'end': Timestamp('2021-08-16 00:00:00', freq='7D'), 'train': 4649162, 'train_users': 850489, 'train_items': 15415, 'test': 279422, 'test_users': 103511, 'test_items': 6698}


...

  0%|          | 0/850489 [00:00<?, ?it/s]


==================== Fold 2 ====================
{'i_split': 2, 'start': Timestamp('2021-08-16 00:00:00', freq='7D'), 'end': Timestamp('2021-08-23 00:00:00', freq='7D'), 'train': 5051815, 'train_users': 906071, 'train_items': 15577, 'test': 298878, 'test_users': 110076, 'test_items': 6679}


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.40402698516845703 seconds
  warnings.warn(


  0%|          | 0/906071 [00:00<?, ?it/s]

,prec@1,recall@1,prec@5,recall@5,prec@10,recall@10,MAP@1,MAP@5,MAP@10,NDCG@1,NDCG@5,NDCG@10,MeanInvUserFreq@1,MeanInvUserFreq@5,MeanInvUserFreq@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,,,,,,,,,,
tfidf_user_knn,0.001833,0.001013,0.001561,0.003745,0.001468,0.006793,0.001013,0.00188,0.00228,0.001013,0.00188,0.00228,11.237014,10.499605,10.203509,0.000021,0.000023,0.000023


## Обучение и сохранение модели

In [29]:
model = CustomUserKnn(model=TFIDFRecommender(num_threads=2), N_users=50)
model.fit(dataset)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0041654109954833984 seconds
  warnings.warn(


  0%|          | 0/83986 [00:00<?, ?it/s]

In [30]:
with open('model.dill', 'wb') as f:
    dill.dump(model, f)

In [31]:
with open('model.dill', 'rb') as f:
    userknn = dill.load(f)

In [35]:
userknn.predict_single(13)

[8, 16, 12, 1176]